<a href="https://colab.research.google.com/github/MuellerLeonard/Word-embeddings_CNN/blob/master/Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu
!pip install keras
!pip install requests
!pip install fasttext

     |████████████████████████████████| 320.4MB 53kB/s 
     |████████████████████████████████| 71kB 4.8MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3017908 sha256=c36aaa72d05e0598e9c8b5bc9b55b93f40dde55b1702f5c1a401d6b4b196810c
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


Data Preprocessing

In [ ]:
import re
import string
from typing import List
import nltk
from nltk import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

def clean_text(tweets: List[str], lang: str = 'english') -> List[str]:
    """
    performes stemming and other "cleanup"
    :param tweets:
    :param lang:
    :return:
    """

    data: List[str] = tweets
    cleaned_text = []
    stops = set(stopwords.words(lang))

    # no ! and .
    table = str.maketrans(dict.fromkeys(""""#$%&()*+,-/:;<=>?@[\\]^_`{|}~"""))

    for text in data:
        text = re.sub(r'https?://[^\s]*', ' tokenlink ', text)
        text = re.sub(r'\.\.\.', ' tokendotdotdot ', text)
        text = re.sub(r'xD', ' tokenxd ', text)
        text = re.sub(r':\)', ' tokenxbrackethappy ', text)
        text = re.sub(r':\(', ' tokenxbracketsad ', text)
        text = re.sub(r':-\)', ' tokennosehappy ', text)
        text = re.sub(r':-\(', ' tokennosesad ', text)
        text = re.sub(r':D', ' tokenxcheer ', text)
        text = re.sub(r':-S', ' tokens ', text)


        ## Convert words to lower case and split them
        text = text.lower().split()

        text = " ".join(text)

        # Clean the text
        text = re.sub(r"(?<!\.)\.(?!\.)", " ", text)
        text = re.sub(r"[^A-Za-z0-9^,!./'+-=]", " ", text)
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r",", " ", text)
        text = re.sub(r"\.", " ", text)
        text = re.sub(r"!", " ! ", text)
        text = re.sub(r"/", " ", text)
        text = re.sub(r"\^", " ^ ", text)
        text = re.sub(r"\+", " + ", text)
        text = re.sub(r"-", " - ", text)
        text = re.sub(r"=", " = ", text)
        text = re.sub(r"'", " ", text)
        text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
        text = re.sub(r":", " : ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r" u s ", " american ", text)
        text = re.sub(r"\0s", "0", text)
        text = re.sub(r" 9 11 ", "911", text)
        text = re.sub(r"e - mail", "email", text)
        text = re.sub(r"j k", "jk", text)
        text = re.sub(r"\s{2,}", " ", text)

        ## Remove puncuation
        text = text.translate(table)

        text = text.split()

        ## Remove stop words[^\s][^\s]
        #text = [w for w in text if not w in stops]

        stemmer = SnowballStemmer(lang)
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
        cleaned_text.append(text)

    return cleaned_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import scipy.io as sio
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
# import cleanup --> already in notebook
import keras
from sklearn.feature_extraction.text import TfidfVectorizer
#from tensorflow.keras.layers import Dense, Embedding, Flatten, Input, Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.utils import np_utils
from tensorflow.keras.preprocessing.sequence import skipgrams, pad_sequences
from sklearn.manifold import TSNE
from tensorflow.keras.models import Sequential
from sklearn import decomposition
#from keras_preprocessing.text import Tokenizer
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import make_sampling_table
from numpy import asarray, zeros
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling3D, GlobalMaxPooling2D, MaxPooling2D, GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.layers import Conv2D, Conv3D, Conv2D, MaxPooling3D, Embedding, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant
# from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model
from scipy import spatial
from gensim.models import Word2Vec
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import concatenate
from tensorflow.keras import backend
import tensorflow as tf
# fastText
#import fasttext.util
import fasttext
from fasttext import load_model
from sklearn.model_selection import train_test_split
import csv
from io import StringIO
import math
from gensim.models import FastText
# 1DConv Net for comparison
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D,Conv2D
from tensorflow.keras.datasets import imdb


def load_data_run_classification():
  # 'data/....npz'
    data = np.load('/content/drive/My Drive/googleColabFiles/sentqs_dataset.npz')
    Xs = data["arr_0"]
    Ys = data["arr_1"]
    Xt = data["arr_2"]
    Yt = data["arr_3"]
    print("Classification Task Test \n")
    from sklearn.ensemble import GradientBoostingClassifier
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(Xs, Ys)
    print(clf.score(Xt, Yt))

def create_tfidf(sen,min_df=10,max_df=100):
    print("Create TF-IDF\n")
    vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df)
    X = vectorizer.fit_transform(sen)
    return X.toarray()

def save_dataset(X, y, prefix =""):
    print("Save Dataset \n")
    y = np.array(y)[:, None]
    dataset = np.concatenate([y,X], axis=1)

    # data/
    np.save("sentqs_da_"+str(prefix)+".npy",dataset)
    return dataset

def seperate_tweets(data,hashtags,sentiment):
    print("Seperate Tweets \n")
    labels = []
    tweets = []
    sentiment_new = []
    for t,s in zip(data,sentiment):
        for h in hashtags:
            t = t.lower()
            h = "#" + h.lower()
            if h in t:
                labels.append(h)
                tweets.append(t.replace(h," "))
                sentiment_new.append(s)
                break

    return labels,tweets,sentiment_new

def generate_data(corpus, window_size, V):
    for words in corpus:
        couples, labels = skipgrams(words, V, window_size, negative_samples=1, shuffle=True,sampling_table=make_sampling_table(V, sampling_factor=1e-05))
        if couples:
            X, y = zip(*couples)
            X = np_utils.to_categorical(X, V)
            y = np_utils.to_categorical(y, V)
            yield X, y

def get_glove_embedding_matrix(texts, dim=200):
    # data/
    if os.path.isfile("sentqs_glove_embedding.npz"):
      # data/
        loaded_embedding = np.load("sentqs_glove_embedding.npz")
        print('Loaded Glove embedding.')
        return loaded_embedding['embedding']
    else:
        # first, build index mapping words in the embeddings set
        # to their embedding vector

        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(texts)
        sequences = tokenizer.texts_to_sequences(texts)

        word_index = tokenizer.word_index

        print('Indexing word vectors.')

        embeddings_index = {}
        # data/
        with open('/content/drive/My Drive/googleColabFiles/dataset/glove.twitter.27B.200d.txt', encoding="utf8") as f:
            for line in f:
                word, coefs = line.split(maxsplit=1)
                coefs = np.fromstring(coefs, 'f', sep=' ')
                embeddings_index[word] = coefs

        print('Found %s word vectors.' % len(embeddings_index))
        print('Preparing embedding matrix.')

        # prepare embedding matrix
        num_words = len(word_index) + 1
        embedding_matrix = np.zeros((num_words, dim))
        counter = 0
        for word, i in word_index.items():
            # if i >= MAX_NUM_WORDS:
            #    counter +=1
            #    continue
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector

        # data/
        np.savez_compressed("sentqs_glove_embedding.npz", embedding=embedding_matrix)
        return embedding_matrix

def get_fastText_embedding_matrix(texts, BASE_DIR = '', MAX_SEQUENCE_LENGTH = 1000, EMBEDDING_DIM = 200):
# dataset dimension must be 200
    FT_MODEL = "/content/drive/My Drive/googleColabFiles/dataset/cc.en200.bin"
    
    #code for reducing dimensions of file cc.en200.bin, path needs to be adjusted
    #FT_MODEL =fasttext.load_model('/content/drive/My Drive/googleColabFiles/dataset/cc.en.300.bin')
    #print("loading model")
    #fasttext.util.reduce_model(FT_MODEL, EMBEDDING_DIM)
    #print("dimensions reduced")

    #FT_MODEL.get_dimension()

    if os.path.isfile("sentqs_fastText_embedding.npz"):
      saved_embedding = np.load("sentqs_fastText_embedding.npz")
      print('loaded pre_trained_fastText embedding.')
      return saved_embedding['embedding']

    else:
      #obtain pretrained vectors:
      print('Indexing word vectors.')

      filename, file_extension = os.path.splitext(FT_MODEL)

      if file_extension == '.vec':
          embeddings_index = {}
          with open(os.path.join(FT_MODEL)) as f:
              for line in f:
                  values = line.split()
                  word = values[0]
                  coefs = np.asarray(values[1:], dtype='float32')
                  embeddings_index[word] = coefs
            
          print('Found %s word vectors.' % len(embeddings_index))

      # use pretrained model on tweets:
      print('Found %s texts.' % len(texts))
      # finally, vectorize the text samples into a 2D integer tensor
      # num_words = size of vocab
      tokenizer = Tokenizer()
      tokenizer.fit_on_texts(texts)
      sequences = tokenizer.texts_to_sequences(texts)
      word_index = tokenizer.word_index
      print('Found %s unique tokens.' % len(word_index))

      print('Preparing embedding matrix.')

      # load the fasttext model
      f = load_model(FT_MODEL)

      # prepare embedding matrix
      num_words = len(word_index) + 1
      emb_matrix = np.zeros((num_words, EMBEDDING_DIM))

      for word, i in word_index.items():
        # if i >= MAX_NUM_WORDS:
          # continue
        # embedding_vector = embeddings_index.get(word)
          embedding_vector = f.get_word_vector(word)
          if embedding_vector is not None:
              # words not found in embedding index will be all-zeros.
              emb_matrix[i] = embedding_vector

      np.savez_compressed("sentqs_fastText_embedding.npz", embedding = emb_matrix)
      return emb_matrix

def get_fastText_sentence_embedding_matrix(text, dim=200):

    if os.path.isfile("sentqs_fastText_sentence_embedding.npz"):
      # data/
        loaded_embedding = np.load("sentqs_fastText_sentence_embedding.npz")
        loaded_embedding = loaded_embedding["embedding"]
        print('Loaded fastText_sentence embedding.')
        return loaded_embedding
    else:
        text = [''.join(x) for x in text]

        #create .txt file from cleaned_tweets
        df = pd.DataFrame(text)
        df.to_csv(r'text.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

        #train unsupervised model with skipgram
        modelft = fasttext.train_unsupervised('text.txt', "skipgram", dim=dim)
        #Model can be saved:
        #modelft.save_model("textmodel.txt")

        #t = Tokenizer()
        #t.fit_on_texts(text)

        #get each sentence vector from fastText Model and create a matrix
        i = 0
        sent_matrix = np.zeros((len(text), dim))
        for sentence in text:
          sentence_vector = modelft.get_sentence_vector(sentence)
          if sentence_vector is not None:
            sent_matrix[i] = sentence_vector
            i = i + 1
            if i == len(text):
              break

        #print("sentence_matrix: ", sent_matrix)
        # data/
        np.savez_compressed("sentqs_fastText_sentence_embedding", embedding=sent_matrix)
        return sent_matrix   

# epochs 100 to 10
# batch_size, epochs depends on how much input data there is
def get_skipgram_sentence_embedding_matrix(text, dim=200, batch_size=64, window_size=5, epochs = 1):
  # data/
    if os.path.isfile("sentqs_skipgram_sentence_embedding.npz"):
      # data/
        loaded_embedding = np.load("sentqs_skipgram_sentence_embedding.npz")
        loaded_embedding = loaded_embedding["embedding"]
        print('Loaded Skipgram embedding.')
        return loaded_embedding
    else:
        text = [''.join(x) for x in text]
        t = Tokenizer()
        t.fit_on_texts(text)
        corpus = t.texts_to_sequences(text)
        #print(corpus)
        V = len(t.word_index)
        step_size = len(corpus) // batch_size
        model = Sequential()
        model.add(Dense(dim, input_dim=V, activation="softmax"))
        model.add(Dense(V, input_dim=dim, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
        model.summary()

        model.fit(generate_data(corpus, window_size, V), epochs=epochs, steps_per_epoch=step_size)
        # model.save("data/sentqs_full_skigram_arc.h5")
        mlb = MultiLabelBinarizer()
        print("step 1")
        enc = mlb.fit_transform(corpus)
        print("step 2")
        emb = enc @ model.get_weights()[0]
        # data/
        np.savez_compressed("sentqs_skipgram_sentence_embedding", embedding=emb)
        return emb

def  get_skipgram_gensim_embedding_matrix(text, dim = 200, window_size=5, min_word_occurance=1, epochs=1):
  # data/
    if os.path.isfile("sentqs_skipgram_gensim_embedding.npz"):
      # data/
        loaded_embedding = np.load("sentqs_skipgram_gensim_embedding.npz")
        loaded_embedding = loaded_embedding["embedding"]
        print('Loaded Skipgram embedding.')
        return loaded_embedding
    else:
        x = [row.split(' ') for row in text]
        model = Word2Vec(x, size=dim, window=window_size, min_count=min_word_occurance, workers=4, sg=1) #sg = 1: use skipgram

        words = model.wv.vocab.keys()
        vocab_size = len(words)
        print("Vocab size", vocab_size)

        t = Tokenizer()
        t.fit_on_texts(text)

        # total vocabulary size plus 0 for unknown words
        # vocab_size = len(vocab) + 1
        # define weight matrix dimensions with all 0
        weight_matrix = zeros((vocab_size, dim))
        # step vocab, store vectors using the Tokenizer's integer mapping
        for word, i in t.word_index.items():
            if i > vocab_size: break
            if word in model.wv.vocab.keys():
                weight_matrix[i] = model.wv[word]

        # data/
        np.savez_compressed("sentqs_skipgram_gensim_embedding", embedding=weight_matrix)
        return weight_matrix

def get_fastText_gensim_embedding_matrix(text, dim = 200, word_ngrams=1, min_char_ngrams=3, max_char_ngrams=6, window_size=5, min_word_occurance=1, epochs=10):
    if os.path.isfile("sentqs_fastText_gensim_embedding.npz"):
      # data/
        loaded_embedding = np.load("sentqs_fastText_gensim_embedding.npz")
        loaded_embedding = loaded_embedding["embedding"]
        print('Loaded fastText_gensim embedding.')
        return loaded_embedding
    else:
        x = [row.split(' ') for row in text]
        modelft = FastText(size=dim, word_ngrams=word_ngrams,min_n=min_char_ngrams, max_n=max_char_ngrams, window=window_size, min_count=min_word_occurance, workers=4) 
        modelft.build_vocab(sentences=x)
        modelft.train(sentences=x, total_examples=len(x), epochs=epochs) #train

        words = modelft.wv.vocab.keys()
        vocab_size = len(words)
        print("Vocab size", vocab_size)

        t = Tokenizer()
        t.fit_on_texts(text)

        # total vocabulary size plus 0 for unknown words
        # vocab_size = len(vocab) + 1
        # define weight matrix dimensions with all 0
        ft_weight_matrix = zeros((vocab_size, dim))
        # step vocab, store vectors using the Tokenizer's integer mapping
        for word, i in t.word_index.items():
            if i > vocab_size: break
            if word in modelft.wv.vocab.keys():
                ft_weight_matrix[i] = modelft.wv[word]

        # data/
        np.savez_compressed("sentqs_fastText_gensim_embedding", embedding=ft_weight_matrix)
        return ft_weight_matrix


def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))


def text_classification_ft(text, sentiment, train_size=0.8, epochs=25, learningrate=1.0, word_n_grams=2, dim=50, bucket=200000, loss='hs'):
    #sentiment: 2.0 = negative, 1.0 = positive, 0.0 = neutral
    #supervised fastText only for Tweets.csv
    #does not have to be Tokenized
    
    #converting float values of sentiment to string values in a new column
    score = [sentiment.size]
    for x in sentiment:
        if x == 0.0:
          score.append("neutral")  
        elif x == 1.0:
          score.append("positive")
        elif x == 2.0:
          score.append("negative")
        
    score.pop(0)

    print(len(score))
    print(score) 

    #Preprocessing
    #combining individual columns to a table
    texts = [''.join(x) for x in text]
    df = pd.DataFrame(texts)
    dfs = pd.DataFrame(sentiment)
    dfsc = pd.DataFrame(score)
    combined = pd.concat([dfs, df, dfsc], axis=1)
    combined.columns = ['sentiment', 'text', 'score']
    combined.to_csv('tweets.csv', index=False)
    tweets = pd.read_csv("tweets.csv")
    
    print(tweets.head())

    #formatting the data-table so that fastText can use it
    col = ['score', 'text']

    tweets = tweets[col]
    tweets['score']=['__label__'+ s for s in tweets['score']]
    tweets['text']= tweets['text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
    tweets.to_csv(r'tweets_updated.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

    print("updated dataframe: ")
    print(tweets.head())
    train_data = math.ceil(len(texts) * train_size)
    test_data = math.floor(len(texts) * (1-train_size))
  
    #split data into head and tail for train and test
    print("splitting data into training and test", train_data, ",", test_data)
    tweets_train = tweets.head(train_data)
    tweets_train.to_csv(r'tweets_train.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
    tweets_test = tweets.tail(test_data)
    tweets_test.to_csv(r'tweets_test.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

    print("train fastText Model supervised with the training data: ")
    # Hyperparameters can be optimized, using the Standard ones
    modelft = fasttext.train_supervised(input="tweets_train.txt")#, lr=learningrate, epoch=epochs, wordNgrams=word_n_grams, bucket=bucket, dim=dim, loss=loss)
    print("words: ", modelft.words)
    print("labels: ", modelft.labels)
    print("Precision and Recall on test data: ")
    print_results(*modelft.test("tweets_test.txt"))

    print("predict on a sentence and return the probability of the labels: ")
    sentence = "i had a bad injury a few months ago, but now i am finally well"
    cleaned_sentence = clean_text(sentence)
    print("sentence: ", cleaned_sentence)
    print(modelft.predict(sentence, k=3))

def generate_embedding_model(text, y,source_idx,target_idx,batch_size=32, epochs = 50, save = True, dim = 200, val_split=0.2,model_size="1D"):
    # Preprocessing
    #MAX_SEQUENCE_LENGTH = len(max(text, key=lambda i: len(i))) + 1
    MAX_SEQUENCE_LENGTH = 335
    texts = [''.join(x) for x in text]
    # finally, vectorize the text samples into a 2D integer tensor
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

    labels = to_categorical(np.asarray(y))
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)
    num_words = len(word_index) + 1
    #for sentence classification
    num_sentences = len(text)

    # split the data into a training set and a validation set
    indices = np.arange(data.shape[0])
    print("indices: ", indices.shape)
    np.random.shuffle(indices)
    data = data[indices]
    print("data: ", data.shape)
    #print("data: ", data)
    labels = labels[indices]
    num_validation_samples = int(val_split * data.shape[0])

    x_train = data[source_idx]
    y_train = labels[source_idx]
    x_val = data[target_idx]
    y_val = labels[target_idx]

    #for sentence classification:
    #1D array, mapping one label to each sentence, like word classification only for sentences --> very clunky, but how i understood it
    #each sentence will get a vector from the sentence_embedding_matrix
    #accuracy around 60%

    number_sentences = len(text)
    matrix = np.arange(number_sentences)
    matrix = matrix[indices]
    w_train = matrix[source_idx]
    v_train = labels[source_idx]
    w_val = matrix[target_idx]
    v_val = labels[target_idx]

    # not sure
    emb = get_skipgram_gensim_embedding_matrix(text, epochs=1)
    emb = np.expand_dims(emb, 1)
    emb_train = emb[:-num_validation_samples]
    emb_val = emb[-num_validation_samples:]
    # Build model
    MAX_SEQUENCE_LENGTH = len(max(text, key=lambda i: len(i))) + 1
    with tf.device('/GPU:0'):
        #sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',name="embedding_input")

        glove_embedding_layer = Embedding(num_words,
                                    dim,
                                    weights=[get_glove_embedding_matrix(texts, dim)],
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=False)#(sequence_input)
        skipgram_embedding_layer = Embedding(num_words,
                                    dim,
                                    weights=[get_skipgram_gensim_embedding_matrix(texts, dim)],
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=False)#(sequence_input)
        #fastText emmbedding layer:
        #pre-Trained model: get_fastText_embedding_matrix
        #only trained on Tweets.csv: get_fastText_gensim_embedding_matrix 
        fastText_embedding_layer = Embedding(num_words,
                                    dim,
                                    weights=[get_fastText_gensim_embedding_matrix(texts, dim)],
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=False)
        #for sentence classification: 
        #FastText: get_fastText_sentence_embedding_matrix 
        #Skip-gram: get_skipgram_sentence_embedding_matrix
        sentence_embedding_layer = Embedding(num_sentences,
                                    dim,
                                    weights=[get_fastText_sentence_embedding_matrix(texts, dim)],
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=False)
        
        #Training with 2 embedding layers at a time
        if model_size =="2D":

            sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
            embedded_sequences = fastText_embedding_layer(sequence_input)
            embedded_sequences1 = skipgram_embedding_layer(sequence_input)
            #FastText: embedded_sequences
            #Glove: glove_embedding_layer
            #Skipgram: skipgram_embedding_layer
            combined=  tf.keras.layers.Lambda(lambda t: tf.stack(t,axis=3))([embedded_sequences, embedded_sequences1])
            x = Conv2D(128, 5, activation='relu')(combined)
            x = MaxPooling2D(5)(x)
            x = Conv2D(128, 5, activation='relu')(x)
            x = MaxPooling2D(5)(x)
            x = Conv2D(128, 5, activation='relu')(x)
            x = GlobalMaxPooling2D()(x)
            x = Dense(128, activation='relu')(x)

        #Model for sentence classification training --> bad
        #kernel size must be 1 and pool_size as well, cause given array is one dimensional     
        if model_size =="sentence":

            sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
            embedded_sequences = sentence_embedding_layer(sequence_input)

            x = Conv1D(64, 1, padding='valid', activation='relu', strides=1) (embedded_sequences)
            x = MaxPooling1D(pool_size=1) (x)
            x = LSTM(70) (x)
            x = Dense(35) (x)
            x = Dense(1) (x)
            x = Activation('sigmoid') (x)

            preds = Dense(3, activation='softmax')(x)
            model = Model(inputs=sequence_input, outputs=preds)

            model.compile(loss='binary_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])
            
            model.summary()
            
            model.fit(w_train, v_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(w_val, v_val))
            return model

        #1DConv Model
        #training with one embedding layer at a time
        #fastText_embedding_layer:
        #glove_embedding_layer:
        #skipgram_embedding_layer:
        if model_size =="1D":
            sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
            embedded_sequences = fastText_embedding_layer(sequence_input)

            x = Conv1D(64, 5, padding='valid', activation='relu', strides=1) (embedded_sequences)
            x = MaxPooling1D(pool_size=4) (x)
            x = LSTM(70) (x)
            x = Dense(35) (x)
            x = Dense(1) (x)
            x = Activation('sigmoid') (x)

        #not used, would be for 3 embeddings at a time
        if model_size =="large":
            skipgram_sentence_embedding = Embedding(num_words,
                                    dim,
                                    #embeddings_initializer=Constant(get_skipgram_gensim_embedding_matrix(text, epochs=1)),
                                    #weights=get_skipgram_gensim_embedding_matrix(text, epochs=1),
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=True)(sequence_input)

            combined = tf.keras.layers.Lambda(lambda t: tf.stack(t,axis=3))([skipgram_embedding_layer, glove_embedding_layer,skipgram_sentence_embedding])
            x = DenseNet121(include_top=False, weights=None, input_shape = (MAX_SEQUENCE_LENGTH, dim, 3))(combined)
            x = GlobalAveragePooling2D()(x)

        #compiling Model
        preds = Dense(3, activation='softmax')(x)
        model = Model(inputs=sequence_input, outputs=preds)
        model.compile(loss='categorical_crossentropy',
                    optimizer='rmsprop',
                    metrics=['acc'])

        model.summary()
        # plot_model(model, to_file='model_combined.png')

        # Train model
        model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(x_val, y_val))
        
        # scores = model.evaluate(x_val, y_val)
        # print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

        if save:
          # data/
            model.save("sentqs_full.h5")
        return model

def tsne_embedding(X):
    print("Starting TSNE\n")
    for p in [5,25,50,75,100]:
        tsne = TSNE(n_components=2, init='random',
             random_state=0, perplexity=p)
        xl = tsne.fit_transform(X)
        # data/
        np.save("sentqs_tsne_"+str(p)+".npy",xl)
        print("Finished TSNE\n")


def describe_dataset(tweets,labels):
    data = pd.DataFrame([tweets, labels]).T
    description = data.describe()
    print(description)

    print("Class Counts:")
    class_counts = data.groupby(1).size()

    x = class_counts.to_numpy()
    keys = class_counts.keys().to_list()
    fig, ax = plt.subplots()
    plt.bar(keys, x)
    plt.ylabel("Tweet Count")
    plt.xticks(range(len(keys)), keys, rotation=45)
    plt.xlabel("Hashtags")
    plt.tight_layout()
    # plots/
    plt.savefig("sentqs_class_dist.pdf", dpi=1000, transparent=True)
    plt.show()

def plot_eigenspectrum(x):
    values = np.linalg.svd(x,compute_uv=False)
    plt.bar(range(101), values[:101], align='center')
    plt.ylabel("Eigenvalue")
    # plt.tight_layout()
    plt.xlabel("No.")
    plt.xticks([0, 20, 40, 60, 80, 100], [1, 20, 40, 60, 80, 100])
    # plots/
    plt.savefig("sentqs_spectra.pdf", transparent=True)
    plt.show()

# Y for fastText
def plot_tsne(X:None,labels):
    tsne_embedding(X)

    y = preprocessing.LabelEncoder().fit_transform(labels)
    for p in [5, 25, 50, 75, 100]:
      # data/
        d = np.load("sentqs_tsne_" + str(p) + ".npy")
        for idx, l in enumerate(list(set(labels))):
            c = np.where(y == idx)[0]
            x = d[c, :]
            plt.scatter(x[:, 0], x[:, 1], s=.5, label=l)
            plt.legend(markerscale=10., bbox_to_anchor=(1, 1.02))
        plt.ylabel("$x_1$")
        plt.xlabel("$x_2$")
        plt.tight_layout()
        # plots/
        plt.savefig('sentqs_tsne_plot_' + str(p) + ".pdf", dpi=1000, transparent=True)
        plt.show()

def create_domain_adaptation_dataset(X,tweets,source_idx,target_idx,sentiment):

    Xs = X[source_idx]
    Xt = X[target_idx]
    Ys = sentiment[source_idx]
    Yt = sentiment[target_idx]
    data = [Xs,Ys,Xt,Yt]
    # data/
    np.savez('sentqs_dataset.npz', *data)
    # data/
    sio.savemat('sentqs_dataset.mat', {'Xs': Xs, 'Xt': Xt, 'Ys': Ys, 'Yt': Yt})
    source_tweets = [tweets[i] for i in source_idx]
    target_tweets = [tweets[i] for i in target_idx]

    # data/
    pd.DataFrame(source_tweets).to_csv("sentqs_source_tweets.csv")
    # data/
    pd.DataFrame(target_tweets).to_csv("sentqs_target_tweets.csv")
    return  Xs,Ys,Xt,Yt

def load_sentqs_tweets():
  # data/
    if os.path.isfile("sentqs_preprocessed.npz"):
      # data/
        loaded_data = np.load("sentqs_preprocessed.npz")
        return loaded_data['cleaned_tweets'],loaded_data["tweets"], loaded_data['y'],loaded_data['sentiment'],loaded_data["source_idx"],loaded_data["target_idx"]
    else:
        hashtags = ['ADBE', 'GOOGL', 'AMZN', 'AAPL', 'ADSK', 'BKNG', 'EXPE', 'INTC', 'MSFT', 'NFLX', 'NVDA', 'PYPL', 'SBUX',
         'TSLA', 'XEL', 'positive', 'bad', 'sad']

        # Loading and preprocessing of tweets
        # only using a small part of available Tweets to not overload the ram
        df = pd.read_csv("/content/drive/My Drive/googleColabFiles/Tweets.csv")
        sentiment = pd.to_numeric(df.iloc[:, -1], errors="raise", downcast="float")
        labels,tweets,sentiment = seperate_tweets(df.iloc[:, 1],hashtags,sentiment)
        cleaned_tweets = clean_text(tweets)

        y = preprocessing.LabelEncoder().fit_transform(labels)

        source_idx,target_idx = create_domain_adaptation_index(tweets,labels,sentiment)
        # data/
        np.savez_compressed("sentqs_preprocessed.npz",tweets=tweets, cleaned_tweets=cleaned_tweets, y=y,sentiment=sentiment,source_idx=source_idx,target_idx=target_idx)
        return cleaned_tweets,tweets, y,sentiment,source_idx,target_idx

def create_domain_adaptation_index(tweets,labels,sentiment):
    labels = np.array([s if "#bad" not in s else "#sad" for s in labels])
    source_idx  = np.array([i for i,val in enumerate(labels) if val== "#sad" or val == "#bad" ])
    target_idx = np.array([i for i,val in enumerate(labels) if val != "#sad" and val != "#bad" ])
    return source_idx,target_idx


def main_preprocessing(mode="multi_semantic_embedding"):

    # Load neccessary informations about the dataset
    cleaned_tweets,tweets,hashtags,sentiment, source_idx, target_idx = load_sentqs_tweets()

    if mode == "multi_semantic_embedding":

        # Obtain embeddings and train deep learning model
        model = generate_embedding_model(cleaned_tweets,sentiment,source_idx,target_idx,model_size="1D")
        # supervised training and testing with fastText on Tweets.csv:
        # text_classification_ft(cleaned_tweets, sentiment)

    elif mode == "train_embedding":
        #Obtain skipgram embedding only
        #Create feature representation: TFIDF-Variants and skipgram embedding with 1000 dimension and negative sampling
        # Output will be saved to disk
        get_glove_embedding_matrix(cleaned_tweets)
        get_skipgram_gensim_embedding_matrix(cleaned_tweets)

        # Sentence Skipgram is the base feature representation of the datatset
        X = get_skipgram_sentence_embedding_matrix(cleaned_tweets)
        create_domain_adaptation_dataset(X,tweets,source_idx,target_idx,sentiment)
        # Another possible embedding:
        # FastText sentence embedding:
        #get_fastText_sentence_embedding_matrix(cleaned_tweets)
        # only trained on tweets.csv:
        get_fastText_gensim_embedding_matrix(cleaned_tweets)
        # pretrained
        get_fastText_embedding_matrix(cleaned_tweets)


    elif mode == "describe_dataset":
        # # Describe dataset with some common characteristics
        describe_dataset(cleaned_tweets,hashtags)

        # ## Plot eigenspectrum of embeddings
        #data/
        X = np.load("sentqs_skipgram_sentence_embedding.npz",allow_pickle=True)
        print(X.files)
        
        plot_eigenspectrum(X['embedding'])

        # ## Plot representation of 2 dimensional tsne embedding
        plot_tsne(X['embedding'],sentiment)

    else:
        ## Loads the data into the program and trains machine learning model
        load_data_run_classification()


if __name__ == '__main__':

    # Obtain the all files of the dataset preprocessing, including plots, feature representation etc.
    # After running this file you will find the corresponding files for classification in the data folder
    # select mode:
    main_preprocessing("multi_semantic_embedding")

Seperate Tweets 

Found 73561 unique tokens.
Shape of data tensor: (61529, 335)
Shape of label tensor: (61529, 3)
indices:  (61529,)
data:  (61529, 335)
Vocab size 73562
Indexing word vectors.
Found 1193514 word vectors.
Preparing embedding matrix.
Loaded Skipgram embedding.
Vocab size 73562
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 335)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 335, 200)          14712400  
_________________________________________________________________
conv1d (Conv1D)              (None, 331, 64)           64064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 82, 64)            0         
_________________________________________________________________
lstm (LSTM)              